In [1]:
import numpy as np
import pandas as pd 
import string
import random
import csv
import time
import math
import multiset 
import plotly.express as px
import json 
import pandas as pd 
import pickle
%load_ext memory_profiler

In [3]:
# import sys

# # These are the usual ipython objects, including this one you are creating
# ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# # Get a sorted list of the objects and their sizes
# sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

## Reading Data Files 

In [7]:
# The target file includes the original sequences that encodes the data. 
df = pd.read_csv("ErlichDesign.csv")
reads = df["sequence"].to_list()
len(reads)
%memit 

peak memory: 107.27 MiB, increment: 0.12 MiB


In [8]:
len(reads[0])

152

In [9]:
# # The errors_shuffled file include the erroneous copies shuffled. 
# with open('errors_shuffled.txt') as f:
#     reads_err = f.readlines()
# for i in range(0,len(reads_err)):
# #     reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
#     reads_err[i] = reads_err[i].strip()
# # reads_err

In [10]:
# np.size(reads_err)


In [11]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyaErlich.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i].strip()
# reads_cl

In [13]:
reads_cl[0]
%memit 

peak memory: 2973.25 MiB, increment: 0.00 MiB


In [ ]:
# for i in range(0,int(len(reads_cl)/8)):
#     reads_cl_cut[i] = reads_cl[i]

# Functions


In [14]:
def hash_fun_ind(x,a,w,l): 
    cnt = 0 
    for i in range(0,len(x)):
        if x[i] == a[cnt]:
            cnt += 1  
        else:
            cnt = 0 
        if cnt == w:
            return i-w+1 
    return -1

def hash_fun(x,a,w,l):
    ind = hash_fun_ind(x,a,w,l)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    bs_str = ''.join(str(e) for e in bs)
    return bs_str
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def merge(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])
    return parent
    
def rep_find(inp,parent):
    temp = inp
    cnt = 0
    while(parent[temp]!=temp and cnt<10):
        cnt += 1
        temp=parent[temp]
    return temp 

def edit_dis(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_in_arr(el,parent):
    cnt = 0
    for i in range(0,len(parent)):
        if parent[i] == el: 
            cnt += 1 
    return cnt 

    
def display_parent(parent):
    clstr = {}
    for i in range(0,len(parent)):
        clstr[i]=[]
    for i in range(0,len(parent)):
        clstr[rep_find(i,parent)].append(i)
    return clstr

def display_parent_ver2(parent,C_til_prev,parent_prev):
    clstr = C_til_prev
    for i in range(0,len(parent)):
        if(parent[i]!=parent_prev[i]):
#                 print(i,rep_find(i,parent_prev),clstr[rep_find(i,parent_prev)])
            clstr[rep_find(i,parent)].extend(clstr[i])
            clstr[rep_find(i,parent_prev)] = []
    return clstr

In [34]:
q =  3
x = "AACCAATACCTTGAACCTAACTCGAGTTAACAAACGCAATTCACAGAACAAGGACGTCGGACGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGACAATGTCAGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT"
y = "AACCAATACCTTGAACCTAACTCGAACAAACGCAATTCACAGAACAAGGACGTCGGACGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGACAATGTCAGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT"
ham_dis(bin_sig(x,q),bin_sig(y,q))

3

In [35]:
edit_dis(x,y)

4

In [41]:
dist=[]
q = 3 
for i in range(0,len(reads)):
    dist.append(ham_dis(bin_sig(reads[0],q),bin_sig(reads[i],q)))
dist.sort()
dist

[0,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,


In [42]:
dist=[]
q = 4 
for i in range(0,len(reads)):
    dist.append(ham_dis(bin_sig(reads[0],q),bin_sig(reads[i],q)))
dist.sort()
dist

[0,
 83,
 86,
 86,
 86,
 87,
 87,
 87,
 87,
 87,
 88,
 88,
 89,
 89,
 89,
 89,
 89,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 92,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 95,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 96,
 

In [46]:
dist=[]
for i in range(0,len(reads)):
    dist.append(edit_dis(reads[0],reads[i]))
    if(i%2000==0):
        print(i)
dist.sort()
dist

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000


[0,
 68,
 69,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 71,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 73,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 

In [56]:
dist=[]
q = 3 
i = 2
while(reads_cl[i]!=""):
    dist.append(ham_dis(bin_sig(reads_cl[0],q),bin_sig(reads_cl[i],q)))
    i +=1
dist.sort()
dist

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 5,
 6]

In [57]:
dist=[]
q = 4 
i = 2
while(reads_cl[i]!=""):
    dist.append(ham_dis(bin_sig(reads_cl[0],q),bin_sig(reads_cl[i],q)))
    i +=1
dist.sort()
dist

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 10,
 11,
 13,
 26,
 40]

In [55]:
dist=[]
q = 4 
i = 2
while(reads_cl[i]!=""):
    dist.append(edit_dis(reads_cl[0],reads_cl[i]))
    i +=1
dist.sort()
dist

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 4,
 5,
 7,
 7,
 12]

In [19]:
%memit 

peak memory: 2973.98 MiB, increment: 0.01 MiB


# Clustering Algorithm

In [20]:
def rep_in_C(read,C_reps):
    lower = 0;
    upper = len(C_reps) - 1;
    while (lower <= upper):
        mid = lower + int((upper - lower) / 2)
#         print(upper,mid)
        res = -1
        if (read == (C_reps[mid][0])):
            return C_reps[mid][1]
        if (read > (C_reps[mid][0])):
            lower = mid + 1;
        else:
            upper = mid - 1;
    return -1;

def comp_clstrs(alg_clstr,org_clstr,gamma):
    num_exist = 0
    if(len(alg_clstr)>len(org_clstr)):
#         print(alg_clstr)
        return 0 
    else:
        for i in range(0,len(alg_clstr)):
            flg_exist = 0
            for j in range(0,len(org_clstr)):
                if(reads_err[alg_clstr[i]] == org_clstr[j]):
                    flg_exist = 1
                    num_exist +=1
                    break
            if(flg_exist == 0):
                return 0
        if(num_exist < gamma*len(org_clstr)):
            return 0
            
        return 1 
    
def calc_acrcy(clustering,C_dict,C_reps,gamma):
#     clustering = display_parent(parent)
    acrcy = 0
    for i in range(0,len(clustering)):
        if(len(clustering[i])>=1):
            acrcy = acrcy + comp_clstrs(clustering[i],C_dict[rep_in_C(reads_err[clustering[i][0]],C_reps)],gamma)
    return acrcy

def min_max(val1,val2):
    min_val = min(val1,val2)
    max_val = max(val1,val2)
    return min_val,max_val
    

In [21]:
# (Read, Cluster rep of the cluster to which the read belongs to)
C_reps = []
C_dict = {}
rep = reads_cl[0]
for i in range(1,len(reads_cl)):
    if reads_cl[i] != "":
        if reads_cl[i][0] == "*":
            if(len(C_reps)>0):
                C_dict[rep].pop()
                C_reps.pop()
            rep=reads_cl[i-1]
            C_dict[rep]=[]
        else:
#             print(C_dict)
            C_dict[rep].append(reads_cl[i])
            C_reps.append((reads_cl[i],rep))
C_reps.sort(key=lambda x:x[0])
%memit 

peak memory: 4017.54 MiB, increment: 0.00 MiB


In [22]:
C_reps[0:5]

[('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAA',
  'AAGGATCTCGATAGATGACGATGACAGCCCTATAAACGCTAGAATAAGGCCAGTAAGAATGGGACCGGCGAGACTCTCATCAAAGCCTTAAGTACCCAAAGACAAACGCAAATGAAGGAATCGAGGACGTCACAGCTGCAACCACAACCATC'),
 ('AAAAAACCCAAACCGATTCCTTGTTCGACTTAGCACGTATTCAGCCTAGATTCACGCTTTCTGAATTCGGCATTGTTCGTAGTCGTTATCAGTGAGATTAATGGGCGCGGCCCGGAGATGTCCCGCAGTGCCAGCTGGTGTAAGCTCAGAA',
  'AAATAAACCCAAACCGATTCCTTGTTCGACTTAGCACGTATTCAGCCTAGATTCACGCTTTCTGAATTCGGCATTGTTCGTAGTCGTTATCAGTGAGATTAATGGGCGCGGCCCGGAGATGTCCCGCAGTGCCAGCTGGTGTAAGCTCAGAA'),
 ('AAAAAACGGTCAAATGGTACTGGGTTGATGCGCCGAAACCATTAAGAATAGCGGTGGGCGTCCTCTTAGGATCATTTCAGGAATTCTTTAGTGTTCGCGGATTGCCGCAATGTTCCCGCGTAATGTGCCTTTACGCCACCCTCACCCG',
  'AAACTCTAAACGGTCAAATGGTACTGGGTTGATGCGCCGAAACCATTAAGAATAGCGGTGGGCGTCCTCTTAGGATCATTTCAGGAATTCTTTAGTGTTCGCGGATTGCCGCAATGTTCCCGCGTAATGTGCCTTTACGCCACCCTCACCCG'),
 ('AAAAAAGTCCATTACGATGTTCGAAAGAGTTTAGGGTAGAGAATGGCGCACT

In [23]:
reads_err = [0]*(len(C_reps))
for i in range(0,len(C_reps)):
    reads_err[i]=C_reps[i][0]
random.shuffle(reads_err)
with open('err_shuffled_erlich.pickle', 'wb') as f:
    pickle.dump(reads_err, f)
%memit

peak memory: 4133.24 MiB, increment: 0.00 MiB


In [24]:
reads_err[0:5]

['TGGCTCAACAGCTACGGGTGGCAGCACTGAACATGAATTGTTAGTCGTGTTAGATTGGCCATATGCGCCTCATCGCTCGGCGGGACTTTGCTTCCGCTCTCAATTTACACACCGTTCTCCCATGACGCCACCGACACACTGGAATAACACAG',
 'AACGGTCAGGGATCGGTCATACCACGATCCAACGACGTATTTCCGTACCGTTGTGTTATCGTAGATCCATAGGACCATGGCTCGGTTGTGGCTCCACCAATTCATTGTGATCTCTCACACAGACTAACCCGTGGGCCGTGAATGTTCGTCTG',
 'GAAGCTTGTAAATTTGTCATACTTCTTGCTCAGATGAGTCTTTGCGCGAACAAAGACGGGATCAAATAAGACCAAGATAGGCTGTCCTCAGTTGTGCGACAGCTTGAAGCGTCGCTGCCCGTTGTGTCCTTGTCGTTGTCCGGTCTGAATGT',
 'TCGAGTCGTGCCGCAAGACCTCACTCAGTCGGACTGCAGCTTATGGATAAGACACAGGGCGTGCTACTATAAATTTCGCCCGTCGCTGAGAACCTCTGTTTGACTCTCCCGAGGCACCTAGCAGAAAATTAGTTGTCTCCAAGCTTGGTGC',
 'TACACCTCATACATGGCGAATACTGAACACACGTAAGACCTTAATACTCATAACTCGTCTTAATCACCGAGTGCCAAGCATAGACAGTCATTGGTGGCAGGGAGCGGGTGTGGGAAAGATGTGGGACCGGCCCTGTCCGAGTTGACGGGTCA']

In [25]:
C_dict[list(C_dict.keys())[0]][0:5]

['AACCAATACCTTGAACCTAACTCGAGTTAACAAACGCAATTCACAGAACAAGGACGTCGGGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGACAATGTCAGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT',
 'AACCAATACCTTGAACCTAACTCGAGTTAACAAACGCAATTCACAGAACAAGGACGTCGGACGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGTCAATGTCAGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT',
 'AACCAATACCTTGAACCTAACTCGAGTTAACAAACGCAATTCACAGAACAAGGACGTCGGACGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGGCAATGTCAGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT',
 'AACCAATACCTTGAACCTAACTCGAGTTAACAAACGCAATTCACAGAACAAGGACGTCGGACGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGACATGTCAGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT',
 'AACCAATACCTTGAACCTAACTCGAGTTAACAAACGCAATTCACAGAACAAGGACGTCGGACGGTGTCCAGAATACCGGCCTCGTGACCGTGGCCAGGGAACCTGACAATGTCTGGCCTTACCGACACACGCAACCTCTTGCTGAAAGGCCT']

In [26]:
# Precompute all the binary signatures
q = 3
bin_sig_arr = []
for i in range(0,len(reads_err)):
    bin_sig_arr.append(bin_sig(reads_err[i],q))
    if(i%400000==0):
        print(i)

# with open('bin_sig_arr_erlich.pickle', 'rb') as f:
#     bin_sig_arr =  pickle.load(f)
    
%memit

0
400000
800000
1200000
1600000
2000000
2400000
2800000
3200000
3600000
4000000
4400000
4800000
5200000
5600000
6000000
6400000
6800000
7200000
7600000
8000000
8400000
8800000
9200000
9600000
10000000
10400000
10800000
11200000
11600000
12000000
12400000
12800000
13200000
peak memory: 5927.15 MiB, increment: 0.01 MiB


In [27]:
with open('bin_sig_arr_erlich.pickle', 'wb') as f:
    pickle.dump(bin_sig_arr, f)

In [59]:
reads_err_ind  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    reads_err_ind[i] = (i,reads_err[i]) 
    parent[i] = i
    
%memit

C_til = display_parent(parent)

%memit

acrcy_dict1 = {}
acrcy_dict2 = {}
acrcy_dict3 = {}
acrcy_dict4 = {}
acrcy_dict5 = {}

time_itr_dict = {}
# parent_dict = {}
# parent_dict[0] = parent.copy() 

# DESIRED_ACRCY = 0.99
gamma = 0.9
card_C = len(reads)
local_comm = 300

q = 3
th_low  = 3
th_high = 5
r = 4
w = math.ceil(math.log(len(reads_err[0]),4))
l = math.ceil(math.log(len(reads_err),4))

lcl_step = 0


for lcl_step in range(0,local_comm):
# while(acrcy < DESIRED_ACRCY):   
    time_start = time.time()

    hash_select = [0]*(len(reads_err))    
    for i in range(0,len(C_til)):
        if(len(C_til[i])!=0):    
            hash_select[random.choice(C_til[i])]=1
            
    a = rand_perm(w)
    hash_C_til = [0]*(len(reads_err))
    for i in range(0,len(reads_err_ind)):
        if(hash_select[i]==0):
            hash_C_til[i] = (i,"")
        else:
            hash_C_til[i] = (i,hash_fun(reads_err_ind[i][1],a,w,l))    
    hash_C_til.sort(key=lambda x:x[1])

    cnt = 0
    for i in range(0,len(hash_C_til)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if hash_C_til[i][1] == hash_C_til[i+1][1]:
#                 if rep_find(i,parent) == rep_find(i+1,parent):
#                     continue 
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_low) or ((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_high) and edit_dis(x,y)<=r)):
                    cnt +=  1                   
                    min_temp,max_temp = min_max(rep_find(hash_C_til[i][0],parent),rep_find(hash_C_til[i+1][0],parent))                    
                    C_til[min_temp].extend(C_til[max_temp])
                    C_til[max_temp] = []        
                    parent[max_temp] = min_temp                    

    if lcl_step%10==7:
        acrcy_dict1[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.6)/card_C
        acrcy_dict2[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.7)/card_C
        acrcy_dict3[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.8)/card_C
        acrcy_dict4[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.9)/card_C
        acrcy_dict5[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,1)/card_C
        print("Accuracy:",acrcy_dict3[lcl_step+1],acrcy_dict5[lcl_step+1] )
#     parent_dict[lcl_step+1]=parent.copy()
    
    time_itr = time.time() - time_start
    time_itr_dict[lcl_step+1]=time_itr
    
    print(time_itr,lcl_step,cnt)
    
#     lcl_step += 1

peak memory: 11622.86 MiB, increment: 0.96 MiB
peak memory: 12080.77 MiB, increment: 0.14 MiB
577.6961028575897 0 4798308
412.7555284500122 1 2790731
315.14275670051575 2 1914769
265.81034779548645 3 1161923
225.16262340545654 4 835631
220.97189354896545 5 607692
189.9200930595398 6 369478
Accuracy: 0.8832916666666667 0.25729166666666664
460.5049443244934 7 226157
117.00162720680237 8 173006
122.62359619140625 9 105060
83.82830286026001 10 1504
108.54617404937744 11 76563
91.0989146232605 12 36769
90.59285402297974 13 33668
89.80331015586853 14 22138
75.49023818969727 15 10497
74.09807062149048 16 4932
Accuracy: 0.9733333333333334 0.4105972222222222
350.01041746139526 17 6773
78.87085461616516 18 5289
10.671039581298828 19 5
75.56483840942383 20 4749
69.05291771888733 21 2050
71.4706449508667 22 2463
69.14107012748718 23 1171
69.2563157081604 24 1243
62.639413833618164 25 629
64.08581113815308 26 534
Accuracy: 0.9879166666666667 0.44227777777777777
355.62566351890564 27 647
65.20404982

KeyboardInterrupt: 

In [62]:
with open('time_itr_dict_5,3,4,82_clstr_dict-Erlich.json', 'w') as fp:
    json.dump(time_itr_dict, fp)
with open('acrcy_dict_5,3,4,82,0.6_clstr_dict-Erlich.json', 'w') as fp:
    json.dump(acrcy_dict1, fp)
with open('acrcy_dict_5,3,4,82,0.7_clstr_dict-Erlich.json', 'w') as fp:
    json.dump(acrcy_dict2, fp)
with open('acrcy_dict_5,3,4,82,0.8_clstr_dict-Erlich.json', 'w') as fp:
    json.dump(acrcy_dict3, fp)
with open('acrcy_dict_5,3,4,82,0.9_clstr_dict-Erlich.json', 'w') as fp:
    json.dump(acrcy_dict4, fp)
with open('acrcy_dict_5,3,4,82,1_clstr_dict-Erlich.json', 'w') as fp:
    json.dump(acrcy_dict5, fp)

In [60]:
total_time = 0
for i in range(1,len(time_itr_dict)+1):
    total_time += time_itr_dict[i]
total_time

9591.1519780159

In [61]:
keys = acrcy_dict1.keys()
values1 = acrcy_dict1.values()
values2 = acrcy_dict2.values()
values3 = acrcy_dict3.values()
values4 = acrcy_dict4.values()
values5 = acrcy_dict5.values()

df = pd.DataFrame()

df["keys"] = keys
df["0.6"]= values1
df["0.7"]= values2
df["0.8"]= values3
df["0.9"]= values4
df["1.0"]= values5

fig = px.line(df, x=df["keys"], y=['0.6', '0.7', '0.8', '0.9', '1.0'])
fig.show()

In [ ]:
keys = time_itr_dict.keys()
values = time_itr_dict.values()

px.line(x=keys,y=values)

In [63]:
!git status 

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Clustering-algorithm-single-core-(multiple-values-of-gamma).ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	Clustering-algorithm-single-core-(gamma=1).ipynb
	Clustering-algorithm-single-core-(q=3,multiple-values-of-gamma)-Erlich.ipynb
	Clustering-algorithm-single-core-(q=4,multiple-values-of-gamma)-Erlich.ipynb
	ErlichDesign.csv
	acrcy_dict_12,8,6,300,0.6_clstr_dict.json
	acrcy_dict_12,8,6,300,0.7_clstr_dict.json
	acrcy_dict_12,8,6,300,0.8_clstr_dict.json
	acrcy_dict_12,8,6,300,0.9_clstr_dict.json
	acrcy_dict_12,8,6,300,1_clstr_dict.json
	acrcy_dict_12,8,6,720,1_clstr_dict.json
	acrcy_dict_5,3,4,82,0.6_clstr_dict-Erlich.json
	acrcy_dict_5,3,4,82,0.7_clstr_dict-Erlich.json
	acrcy_dict_5,3,4,82,0.8_clstr_dict-Erlich.json
	acrcy_dict_5,

In [ ]:
!git add Clustering-algorithm-single-core-\(multiple-values-of-gamma\).ipynb

In [ ]:
# !git add -u && git status && git commit -m "clstr_dict and parent_arr approaches" && git push origin master 